# Implementation of Wasserstein GANs
Reference: https://arxiv.org/pdf/1701.07875.pdf

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

In [3]:
def to_cuda(x):
    x = Variable(x)
    if torch.cuda.is_available():
        return x.cuda()
    return x

In [4]:
def to_onehot(x, num_classes=10):
    assert isinstance(x, int) or isinstance(x.data, (torch.cuda.LongTensor, torch.LongTensor))    
    if isinstance(x, int):
        c = to_cuda(torch.zeros(1, num_classes))
        c.data[0][x] = 1
    else:
        c = to_cuda(torch.FloatTensor(x.size(0), num_classes))
        c.zero_()
        c.scatter_(1, x, 1) # dim, index, src value
    return c

In [5]:
def n_critic(step, nc=5):
    if step < 25 or step % 500 == 0:
        return 100
    return nc

In [6]:
class Discriminator(nn.Module):
    """
        Convolutional Discriminator for MNIST
    """
    def __init__(self, input_size=784, num_classes=1):
        super(Discriminator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 200),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(200, num_classes),
        )
    
    def forward(self, x):
        y_ = x.view(x.size(0), -1)
        y_ = self.layer1(y_)
        y_ = self.layer2(y_)
        y_ = self.layer3(y_)
        return y_

In [7]:
class Generator(nn.Module):
    """
        Convolutional Generator for MNIST
    """
    def __init__(self, input_size=100, num_classes=784):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 200),
            nn.ReLU(),
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Linear(200, num_classes),
            nn.Tanh()
        )
        
    def forward(self, x):
        y_ = self.layer(x)
        y_ = y_.view(x.size(0), 1, 28, 28)
        return y_

In [8]:
D = Discriminator().cuda()
G = Generator().cuda()

In [9]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))]
)

In [10]:
mnist = datasets.MNIST(root='../data/', train=True, transform=transform, download=True)

In [11]:
batch_size = 64

In [12]:
data_loader = DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True, drop_last=True)

In [13]:
D_opt = torch.optim.RMSprop(D.parameters(), lr=0.00005)
G_opt = torch.optim.RMSprop(G.parameters(), lr=0.00005)

In [14]:
max_epoch = 100 # need more than 100 epochs for training generator
step = 0
g_step = 0
n_noise = 100

In [15]:
D_labels = to_cuda(torch.ones(batch_size)) # Discriminator Label to real
D_fakes = to_cuda(torch.zeros(batch_size)) # Discriminator Label to fake

In [16]:
for epoch in range(max_epoch):
    for idx, (images, labels) in enumerate(data_loader):
        step += 1
                   
        # Training Discriminator
        x = to_cuda(images)
        x_outputs = D(x)

        z = to_cuda(torch.randn(batch_size, n_noise))
        z_outputs = D(G(z))
        D_x_loss = torch.mean(x_outputs)
        D_z_loss = torch.mean(z_outputs)
        D_loss = (D_x_loss - D_z_loss) * -1
        
        D.zero_grad()
        D_loss.backward()
        D_opt.step()
        # Parameter Clipping
        for p in D.parameters():
            p.data.clamp_(-0.01, 0.01)
                    
        if step % n_critic(g_step) == 0:
            g_step += 1
            # Training Generator
            z = to_cuda(torch.randn(batch_size, n_noise))
            z_outputs = D(G(z))
            G_loss = -torch.mean(z_outputs, 0)

            D.zero_grad()
            G.zero_grad()
            G_loss.backward()
            G_opt.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {}'.format(epoch, max_epoch, step, -D_loss.data[0], -G_loss.data[0]))

Epoch: 1/100, Step: 1000, D Loss: 8.951957702636719, G Loss: 0.5812785029411316
Epoch: 2/100, Step: 2000, D Loss: 7.873332500457764, G Loss: 1.6911933422088623
Epoch: 3/100, Step: 3000, D Loss: 0.7095146179199219, G Loss: 6.424520492553711
Epoch: 4/100, Step: 4000, D Loss: 0.26197004318237305, G Loss: 4.833785533905029
Epoch: 5/100, Step: 5000, D Loss: 0.011798381805419922, G Loss: 4.168814659118652
Epoch: 6/100, Step: 6000, D Loss: 0.10111021995544434, G Loss: 3.087111234664917
Epoch: 7/100, Step: 7000, D Loss: 0.14515209197998047, G Loss: 2.2805087566375732
Epoch: 8/100, Step: 8000, D Loss: -0.04920172691345215, G Loss: 2.0345091819763184
Epoch: 9/100, Step: 9000, D Loss: -0.01527559757232666, G Loss: 1.6543197631835938
Epoch: 10/100, Step: 10000, D Loss: -0.04245328903198242, G Loss: 1.3323910236358643
Epoch: 11/100, Step: 11000, D Loss: -0.05783283710479736, G Loss: 1.2323732376098633
Epoch: 12/100, Step: 12000, D Loss: -0.04268360137939453, G Loss: 1.183617353439331
Epoch: 13/100,

In [17]:
import scipy.misc

In [32]:
# generation to image
z = to_cuda(torch.randn(1, n_noise))
fakeimg = G(z).view(28, 28)
img = fakeimg.cpu().data.numpy()
scipy.misc.toimage(img)

/home/yangyangii/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  """


In [15]:
def save_checkpoint(state, file_name='checkpoint.pth.tar'):
    torch.save(state, file_name)

In [16]:
# Saving params.
# torch.save(D.state_dict(), 'D_c.pkl')
# torch.save(G.state_dict(), 'G_c.pkl')
save_checkpoint({'epoch': epoch + 1, 'state_dict':D.state_dict(), 'optimizer' : D_opt.state_dict()}, 'D_was.pth.tar')
save_checkpoint({'epoch': epoch + 1, 'state_dict':G.state_dict(), 'optimizer' : G_opt.state_dict()}, 'G_was.pth.tar')